# Change runtime until you get P4

In [ ]:
!nvidia-smi
# if you see K80 change the runtime until you see P4.

Wed Jul  8 16:24:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name not in [b'Tesla T4', b'Tesla P4']:
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!', device_name)

In [ ]:
# cuda version
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# python version
!python -V; pip -V

Python 3.6.9
pip 19.3.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


# Install cuda
- https://news.developer.nvidia.com/run-rapids-on-google-colab/

In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 165 (delta 60), reused 20 (delta 4), pack-reused 0
Receiving objects: 100% (165/165), 48.48 KiB | 670.00 KiB/s, done.
Resolving deltas: 100% (60/60), done.
PLEASE READ
********************************************************************************************************
Changes:
1. Default stable version is now 0.14.  Nightly is now 0.15.  We have fixed the long conda install.  Hooray!
2. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.15, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
     

In [ ]:
import cudf
import pandas as pd

import pynvml
import numpy as np
import xgboost as xgb


#load data from skl, then split it into testing and training data
## Load data
from sklearn.datasets import load_boston
boston = load_boston()
pdata = pd.DataFrame(boston.data)
data = cudf.from_pandas(pdata)

## spliting training and test set
from cuml import train_test_split
X, y = data.iloc[:,:-1],data.iloc[:,12]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [ ]:
# XGBoost Time!
## Create train and test dmatrix
dtrain = xgb.DMatrix(
        X_train,
        y_train    )

dtest = xgb.DMatrix(
        X_test,
        y_test    )

## Train the model
trained_model = xgb.train(
                        {
                          'learning_rate': 0.1,
                          'colsample_bytree' : 0.3,
                          'max_depth': 5,
                          'objective': 'reg:linear',
                          'n_estimators':10,
                          'alpha' : 10,
                          'silent': True,
                          'verbose_eval': True,
                          'tree_method':'gpu_hist',
                        },
                        dtrain,
                        num_boost_round=100, evals=[(dtrain, 'train')])

## Predict the model
prediction = trained_model.predict(dtest)

[16:42:55] WARNING: /conda/conda-bld/xgboost_1591204502249/work/src/objective/regression_obj.cu:168: reg:linear is now deprecated in favor of reg:squarederror.
[16:42:55] WARNING: /conda/conda-bld/xgboost_1591204502249/work/src/learner.cc:480: 
Parameters: { n_estimators, silent, verbose_eval } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:13.15440
[1]	train-rmse:12.07691
[2]	train-rmse:11.10576
[3]	train-rmse:10.23103
[4]	train-rmse:9.48240
[5]	train-rmse:8.79232
[6]	train-rmse:8.14586
[7]	train-rmse:7.63540
[8]	train-rmse:7.13521
[9]	train-rmse:6.69863
[10]	train-rmse:6.21223
[11]	train-rmse:5.88771
[12]	train-rmse:5.58643
[13]	train-rmse:5.29987
[14]	train-rmse:5.08797
[15]	train-rmse:4.90648
[16]	train-rmse:4.71301
[17]	train-rmse:4.45847
[18]	train-rmse:4.3

In [ ]:
# Form and test predictions from xgboost output
## MSE requires values be float32 
y_test = y_test.astype(np.float32) 

## Test prediction wih RMSE, compare it to sklearn and pandas.  
from cuml.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, prediction))

## RMSE for the price prediction is per 1000$.  Let's see what we get...
print("RMSE: ", rmse)

RMSE:  3.1533444
